In [4]:
# !pip install PyPDF2
# !pip install pdfplumber

In [2]:
import PyPDF2
import requests
import pdfplumber

In [ ]:
# Откройте PDF-файл
with open('Class_2/Приказ_Министерства_здравоохранения_Российской_Федерации_от_7_ноября_2012_г._№_666н.pdf', 'rb') as pdf_file:
    # Создайте объект PDF-документа
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Получите количество страниц в документе
    num_pages = len(pdf_reader.pages)

    # Прочитайте содержимое каждой страницы
    for page_number in range(num_pages):
        page = pdf_reader.pages[page_number]
        page_text = page.extract_text().split(" ")
        print(page_text)
        print("\n\n\n\n\n\n")

In [6]:
def get_mkb_codes(path):
    with open(path, 'rb') as pdf_file:
        # Создайте объект PDF-документа
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Получите количество страниц в документе
        num_pages = len(pdf_reader.pages)
        page = pdf_reader.pages[1]
        data_pages_1_2 = page.extract_text().replace("\n", "").split(" ")
        page = pdf_reader.pages[2]
        page_text = page.extract_text().replace("\n", "").split(" ")
        data_pages_1_2.extend(page_text)
        if num_pages > 3:
            page = pdf_reader.pages[3]
            page_text = page.extract_text().replace("\n", "").split(" ")
            data_pages_1_2.extend(page_text)
            if num_pages > 4:
                page = pdf_reader.pages[4]
                page_text = page.extract_text().replace("\n", "").split(" ")
                data_pages_1_2.extend(page_text)
                if num_pages > 5:
                    page = pdf_reader.pages[5]
                    page_text = page.extract_text().replace("\n", "").split(" ")
                    data_pages_1_2.extend(page_text)
        if "МКБ" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБ")
        elif "МКБX*(1)" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБX*(1)")
        elif "МКБX*" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБX*")
        elif "МКБX" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБX")
        elif "МКБХ*(1)" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБХ*(1)")
        elif "МКБХ*" in data_pages_1_2:
            start_point = data_pages_1_2.index("МКБХ*")
        else:
            start_point = data_pages_1_2.index("МКБХ")
        end_point = data_pages_1_2.index("услуги", start_point)
        data_pages_1_2 = data_pages_1_2[(start_point + 1) : end_point]

        mkb_list = list()
        for elem in data_pages_1_2:
            elem = elem.strip()
            if len(elem) > 1 and len(elem) < 8:
                if elem[1].isnumeric() and not elem[0].isnumeric():
                    mkb_list.append(elem)
        return mkb_list

In [9]:
def get_avaliable_of_tables(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf"):
    with open(path, 'rb') as pdf_file:
        flag_1 = 0
        flag_2 = 0
        table_page_pos = [0,0]
        # Создайте объект PDF-документа
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Получите количество страниц в документе
        num_pages = len(pdf_reader.pages)

        # Прочитайте содержимое каждой страницы
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            page_text = page.extract_text()
            if "Медицинские услуги для лечения заболевания" in page_text:
                flag_1 = 1
                table_page_pos[0] = page_number
            if "Перечень лекарственных препаратов для медицинского применения" in page_text:
                flag_2 = 1
                table_page_pos[1] = page_number
        if flag_1 == 0 and flag_2 == 0:
            return [1, table_page_pos]
        elif flag_1 == 1 and flag_2 == 0:
            return [2, table_page_pos]
        elif flag_1 == 0 and flag_2 == 1:
            return [3, table_page_pos]
        else:
            return [4, table_page_pos]

In [12]:
def check_boundary_conditions(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf", 
                              table_var_number = 2, page_pos = answer[1][1]):
    with open(path, 'rb') as pdf_file:
        flag = False
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        if table_var_number == 1:
            page = pdf_reader.pages[page_pos+1]
            page_text = page.extract_text().split("\n")
            if "Прием" in page_text[0]:
                flag = True
        elif table_var_number == 2:
            page = pdf_reader.pages[page_pos+1]
            page_text = page.extract_text().split("\n")
            if "Код" in page_text[0] or "Российской Федерации" in page_text[0]:
                flag = True
    return flag

In [13]:
check_boundary_conditions(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf", 
                              table_var_number = 2, page_pos = answer[1][1])

False

In [16]:
# new_index = 0
#         if len(table) != 0:
#             if not list_data:
#                 for col in table[0].extract():
#                     for row in col:
#                         if row:
#                             if "МКБ" in row:
#                                 new_index = 1
#             if new_index==0 or (new_index==1 and len(table) > 1):
#                 list_data += table[new_index].extract()

In [14]:
def get_data_case_1(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_2_февраля_2015 г._№ 31н.pdf"):
    pdf=pdfplumber.open(path)
    pages = pdf.pages
    page_count = len(pages)
    list_data = list()
    for i in range(page_count):
        table=pdf.pages[i].find_tables()
        if len(table) != 0:
            list_data += table[0].extract()
    return list_data

In [19]:
def get_data_case_2(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf", pages_number_1 = 2):
    page_pos = pages_number_1
    if check_boundary_conditions(path, table_var_number = 1, page_pos = pages_number_1):
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            if i != page_pos:
                table=pdf.pages[i].find_tables()
                if len(table) != 0:
                    list_data += table[0].extract()
            else:
                table=pdf.pages[i].find_tables()
                if len(table) > 1:
                    list_data += table[0].extract()
                elif len(table) == 1:
                    pass            
        return list_data
    else:
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            table=pdf.pages[i].find_tables()
            if len(table) != 0:
                list_data += table[0].extract()         
        return list_data

In [24]:
def get_data_case_3(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf", pages_number_2 = 2):
    page_pos = pages_number_2
    if check_boundary_conditions(path, table_var_number = 2, page_pos = pages_number_2):
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            if i != page_pos:
                table=pdf.pages[i].find_tables()
                if len(table) != 0:
                    list_data += table[0].extract()
            else:
                table=pdf.pages[i].find_tables()
                if len(table) > 1:
                    list_data += table[0].extract()
                elif len(table) == 1:
                    pass            
        return list_data
    else:
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            table=pdf.pages[i].find_tables()
            if len(table) != 0:
                list_data += table[0].extract()         
        return list_data

In [25]:
def get_data_case_4(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1579н.pdf", pages_number_1 = 2, pages_number_2 = 2):
    if pages_number_1 == pages_number_2:
        page_pos = pages_number_1
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            if i != page_pos:
                table=pdf.pages[i].find_tables()
                if len(table) != 0:
                    list_data += table[0].extract()
            else:
                table=pdf.pages[i].find_tables()
                if len(table) > 2:
                    list_data += table[0].extract()
    else:
        page_pos = pages_number_1
        pdf=pdfplumber.open(path)
        pages = pdf.pages
        page_count = len(pages)
        list_data = list()
        for i in range(page_pos+1):
            if i != page_pos:
                table=pdf.pages[i].find_tables()
                if len(table) != 0:
                    list_data += table[0].extract()
            else:
                table=pdf.pages[i].find_tables()
                if len(table) > 1:
                    list_data += table[0].extract()
    return list_data

In [27]:
exceptions = []

In [28]:
def get_list_from_pdf(path):
    try:
        flag, position_page = get_avaliable_of_tables(path)
        if flag == 1:
            data_case = get_data_case_1()
        elif flag == 2:
            data_case = get_data_case_2(path = path, pages_number_1 = position_page[0])
        elif flag == 3:
            data_case = get_data_case_3(path = path, pages_number_2 = position_page[1])
        else:
            data_case = get_data_case_4(path = path, pages_number_1 = position_page[0], pages_number_2 = position_page[1])
        data_case_clear = []
        for value in data_case:
            if value[2] != None and value[2].replace(",", "").replace(".", "").isnumeric():
                data_case_clear.append(value)
        return data_case_clear
    except:
        exceptions.append(path)
        return None

In [ ]:
get_list_from_pdf(path = "Приказ_Министерства_здравоохранения_Российской_Федерации_от_9_ноября_2012_г._№_808н.pdf")

In [30]:
import requests 
from bs4 import BeautifulSoup 
 
# URL of the HTML page 
url = "https://minzdrav.gov.ru/ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi" 
 
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)

response = requests.get(url, headers=headers)
html_content = response.text 
 
# Parse the HTML content using BeautifulSoup 
soup = BeautifulSoup(html_content, "html.parser") 
 
# Find all anchor tags with href attributes starting with "sss.qq" 
href_values = [] 
for anchor in soup.find_all("a", href=True): 
    href = anchor["href"] 
    if href.startswith("/ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass"): 
        href_values.append(href) 


# print(len(href_values))
# for href in href_values: 
#     href = "https://minzdrav.gov.ru/" + href
    
href_list_class = ["https://minzdrav.gov.ru/" + x for x in href_values]

In [31]:
href_list_class

['https://minzdrav.gov.ru//ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass-i-nekotorye-infektsionnye-i-parazitarnye-bolezni-a00-b99',
 'https://minzdrav.gov.ru//ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass-ii-novoobrazovaniya-c00-d48',
 'https://minzdrav.gov.ru//ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass-iii-bolezni-krovi-krovetvornyh-organov-i-otdelnye-narusheniya-vovlekayuschie-immunnyy-mehanizm-d50-d89',
 'https://minzdrav.gov.ru//ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass-iv-bolezni-endokrinnoy-sistemy-rasstroystva-pitaniya-i-narusheniya-obmena-veschestv-e00-e90',
 'https://minzdrav.gov.ru//ministry/61/22/stranitsa-979/stranitsa-983/2-standarty-spetsializirovannoy-meditsinskoy-pomoschi/klass-v-psihicheskie-rasstroystva-i-rasstroystva-povedeniya-f00-f99'

In [32]:
def get_href_values(href_elem):    

    url = href_elem

    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'My User Agent 1.0',
        }
    )

    response = requests.get(url, headers=headers)
    html_content = response.text 

    # Parse the HTML content using BeautifulSoup 
    soup = BeautifulSoup(html_content, "html.parser") 

    # Find all anchor tags with href attributes starting with "sss.qq" 
    href_values_1 = [] 
    for anchor in soup.find_all("a", href=True): 
        href = anchor["href"] 
        if href.startswith("http://minzdrav.gov.ru/documents"): 
            href_values_1.append(href)
    # print(href_values_1)
    return href_values_1

In [33]:
file_path_names_dict = dict()

In [34]:
def get_download_link(href_values):
    download_links = []
    for link in href_values:
        url = link

        headers = requests.utils.default_headers()

        headers.update(
            {
                'User-Agent': 'My User Agent 1.0',
            }
        )
        response = requests.get(url, headers=headers)
        html_content = response.text 

        # Parse the HTML content using BeautifulSoup 
        soup = BeautifulSoup(html_content, "html.parser") 

        # Find all anchor tags with a 'download' parameter
        for anchor in soup.find_all("a", download=True, href=True): 
            download_links.append(["https:" + anchor["href"], anchor["download"]]) 
            
    return download_links
        # # Print the found download values 
        # for download in download_links: 
        #     print("https:" + download)

In [35]:
def parcer_all_data(href_list_class, class_number):
    all_download_links_value = get_download_link(href_values = get_href_values(href_elem = href_list_class[class_number-1]))
    for elem in all_download_links_value:
        # print(elem)
        with open("Class_" + str(class_number) + "\\" + elem[1], "wb") as file:
            url = elem[0]
            headers = requests.utils.default_headers()
            headers.update(
                {
                    'User-Agent': 'My User Agent 1.0',
                }
            )
            response = requests.get(url, headers=headers)
            file.write(response.content)
            print(elem[1])
    return all_download_links_value

In [57]:
print_value = parcer_all_data(href_list_class = href_list_class, class_number = 5)

In [ ]:
print_value

In [52]:
numb = 5

for elem in print_value:
    if numb not in file_path_names_dict.keys():
        file_path_names_dict[numb] = [elem[1]]
    else:
        file_path_names_dict[numb].append(elem[1])

In [ ]:
file_path_names_dict

In [36]:
import pickle

# # Сохраняем список в файл
# with open('pdf_names_prikaz_dict.pkl', 'wb') as f:
#     pickle.dump(file_path_names_dict, f)


# Загружаем список из файла
with open('pdf_names_prikaz_dict.pkl', 'rb') as f:
    file_path_names_dict = pickle.load(f)



In [ ]:
file_path_names_dict

In [ ]:
print_value[0]

In [314]:
total_dict = dict()

In [ ]:
cnt_123 = 1

for i in range(1,22):
    for cnt_1 in file_path_names_dict[i]:
        print(cnt_123, i, cnt_1)
        list_of_mkb__1 = get_mkb_codes("Class_" + str(i) + "\\" + cnt_1)
        mkb_data_list = get_list_from_pdf("Class_" + str(i) + "\\" + cnt_1)
        if mkb_data_list:
            for smth_mkb in list_of_mkb__1:
                total_dict[smth_mkb] = mkb_data_list
        cnt_123 += 1

In [ ]:
total_dict

In [316]:
exceptions

['Class_1\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_9_ноября_2012_г._№_803н.pdf',
 'Class_1\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_9_ноября_2012_г._№_803н.pdf',
 'Class_1\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_28_декабря_2012_г._№_1582н.pdf',
 'Class_1\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_9_ноября_2012_г._№_803н.pdf',
 'Class_5\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_4_сентября_2012_г._№_127н.pdf',
 'Class_5\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_4_сентября_2012_г._№_135н.pdf',
 'Class_10\\Приказ_Министерства_здравоохранения_Российской_Федерации_от_9_ноября_2012_г.№_724н.pdf']

In [39]:
import pickle

# Загружаем список из файла
with open('pdf_data_prikaz_dict.pkl', 'rb') as f:
    total_dict = pickle.load(f)

In [40]:
len(total_dict)

1104

In [ ]:
counter = 0
all_docs_list = list()
for key in file_path_names_dict:
    print(key)
    intersection = list(set(all_docs_list).intersection(file_path_names_dict[key]))
    if len(intersection) != 0:
        print(key, intersection)
    all_docs_list += list(set(file_path_names_dict[key]))
    counter += len(set(file_path_names_dict[key]))
print(counter)

In [82]:
len(list(set(all_docs_list)))

430

In [88]:
temp_dict = {}

In [84]:
print_value = parcer_all_data(href_list_class = href_list_class, class_number = 5)

In [ ]:
cnt_123 = 1

for i in range(1,22):
    print_value = parcer_all_data(href_list_class = href_list_class, class_number = i)
    print(cnt_123)
    for cnt_1 in print_value:
        # print(cnt_123, i, cnt_1)
        list_of_mkb__1 = get_mkb_codes("Class_" + str(i) + "\\" + cnt_1[1])
        # mkb_data_list = get_list_from_pdf("Class_" + str(i) + "\\" + cnt_1)
        if len(list_of_mkb__1) != 0:
            for smth_mkb in list_of_mkb__1:
                if smth_mkb in temp_dict:
                    temp_dict[smth_mkb].append(cnt_1)
                else:
                    temp_dict[smth_mkb] = [cnt_1]
        else:
            print(f"ERROR!!!!!! cnt_1")
        cnt_123 += 1

In [46]:
temp_dict["I10"]

NameError: name 'temp_dict' is not defined

In [95]:
# # Сохраняем список в файл
# with open('pdf_prikaz_mkb_codes_href_dict.pkl', 'wb') as f:
#     pickle.dump(temp_dict, f)

In [1]:
temp_dict

NameError: name 'temp_dict' is not defined